<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/03_PyTorch_Fundamentals_Medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. Fondamentaux PyTorch pour Applications Médicales

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Maîtriser les outils PyTorch essentiels pour l'intelligence artificielle médicale.

**Applications pratiques :**
- Manipulation de données patients avec les tenseurs
- Intégration des modèles HuggingFace en contexte médical
- Traitement d'images médicales par lots
- Utilisation de modèles pré-entraînés pour l'inférence

**Important:** Ce cours vous enseigne l'utilisation pratique de PyTorch, pas la programmation avancée.

## Installation et Configuration

In [ ]:
# Installation des bibliothèques essentielles pour l'IA médicale
!pip install torch torchvision transformers datasets huggingface_hub -q
!pip install pandas numpy matplotlib seaborn scikit-learn -q
!pip install pydicom pillow -q

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Configuration pour la reproductibilité
torch.manual_seed(42)
np.random.seed(42)

# Détection du dispositif de calcul
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositif utilisé: {device}")
if torch.cuda.is_available():
    print(f"GPU détecté: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("Configuration PyTorch médical terminée.")

## 1. Tenseurs PyTorch pour Données Médicales

Les tenseurs PyTorch sont la structure de données fondamentale pour l'IA médicale. Ils permettent de manipuler efficacement les données patients, images médicales et paramètres biologiques.

In [ ]:
# Exemple 1: Données de cohorte de patients
print("=== GESTION DE COHORTES DE PATIENTS ===")

# Simuler des données de 100 patients
n_patients = 100

# Caractéristiques démographiques et cliniques
# [âge, poids_kg, taille_m, tension_systolique, tension_diastolique, glycémie]
patient_data = torch.tensor([
    [np.random.normal(65, 15), np.random.normal(70, 12), np.random.normal(1.70, 0.15),
     np.random.normal(130, 20), np.random.normal(80, 10), np.random.normal(5.5, 1.2)]
    for _ in range(n_patients)
], dtype=torch.float32)

print(f"Cohorte de patients: {patient_data.shape}")
print(f"Caractéristiques: Âge, Poids(kg), Taille(m), TA_sys, TA_dia, Glycémie")

# Calculs vectorisés d'indices cliniques
ages = patient_data[:, 0]
poids = patient_data[:, 1]
tailles = patient_data[:, 2]
ta_sys = patient_data[:, 3]
ta_dia = patient_data[:, 4]
glycemie = patient_data[:, 5]

# Calcul de l'IMC pour toute la cohorte
imc = poids / (tailles ** 2)

# Pression artérielle moyenne
pam = ta_dia + (ta_sys - ta_dia) / 3

# Catégorisation automatique
obesite = imc > 30
hypertension = ta_sys > 140
diabete = glycemie > 7.0

print(f"\nRésultats de la cohorte:")
print(f"IMC moyen: {imc.mean():.1f} ± {imc.std():.1f}")
print(f"Patients obèses: {obesite.sum().item()}/{n_patients} ({obesite.float().mean()*100:.1f}%)")
print(f"Patients hypertendus: {hypertension.sum().item()}/{n_patients} ({hypertension.float().mean()*100:.1f}%)")
print(f"Patients diabétiques: {diabete.sum().item()}/{n_patients} ({diabete.float().mean()*100:.1f}%)")

In [ ]:
# Exemple 2: Matrices de corrélation clinique
print("=== ANALYSE DE CORRÉLATIONS CLINIQUES ===")

# Calcul de la matrice de corrélation
correlation_matrix = torch.corrcoef(patient_data.T)

# Visualisation avec matplotlib
features = ['Âge', 'Poids', 'Taille', 'TA_sys', 'TA_dia', 'Glycémie']

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix.numpy(), annot=True, cmap='coolwarm', center=0,
            xticklabels=features, yticklabels=features, fmt='.2f')
plt.title('Matrice de Corrélation - Paramètres Cliniques')
plt.tight_layout()
plt.show()

# Identification des corrélations significatives
print("\nCorrélations cliniques notables:")
for i in range(len(features)):
    for j in range(i+1, len(features)):
        corr_val = correlation_matrix[i, j].item()
        if abs(corr_val) > 0.3:  # Seuil de corrélation modérée
            print(f"{features[i]} ↔ {features[j]}: r = {corr_val:.3f}")

## 2. Images Médicales et Tenseurs

Les images médicales nécessitent une manipulation spécialisée. PyTorch offre des outils efficaces pour le traitement par lots d'images radiologiques.

In [ ]:
# Création d'images médicales simulées
print("=== TRAITEMENT D'IMAGES MÉDICALES ===")

def create_medical_image_batch(batch_size=8, image_size=256):
    """
    Crée un lot d'images médicales simulées pour démonstration
    Format: (batch_size, channels, height, width)
    """
    images = torch.zeros(batch_size, 1, image_size, image_size)
    
    for i in range(batch_size):
        # Base d'image (tissu normal)
        base_intensity = 0.3 + torch.rand(1) * 0.2
        images[i, 0] = base_intensity
        
        # Ajout de structures anatomiques
        center_x, center_y = image_size // 2, image_size // 2
        
        # Structure circulaire (organe)
        y_coords, x_coords = torch.meshgrid(torch.arange(image_size), torch.arange(image_size), indexing='ij')
        distance = torch.sqrt((x_coords - center_x)**2 + (y_coords - center_y)**2)
        organ_mask = distance < (30 + torch.rand(1) * 20)
        images[i, 0][organ_mask] = 0.7 + torch.rand(1) * 0.2
        
        # Anomalie potentielle (probabilité 30%)
        if torch.rand(1) < 0.3:
            anomaly_x = torch.randint(50, image_size-50, (1,)).item()
            anomaly_y = torch.randint(50, image_size-50, (1,)).item()
            anomaly_size = torch.randint(10, 25, (1,)).item()
            
            anomaly_distance = torch.sqrt((x_coords - anomaly_x)**2 + (y_coords - anomaly_y)**2)
            anomaly_mask = anomaly_distance < anomaly_size
            images[i, 0][anomaly_mask] = 0.9 + torch.rand(1) * 0.1
    
    return images

# Génération du lot d'images
medical_batch = create_medical_image_batch(batch_size=8, image_size=256)
print(f"Lot d'images médicales créé: {medical_batch.shape}")
print(f"Format: [batch, canaux, hauteur, largeur]")
print(f"Plage d'intensité: [{medical_batch.min():.3f}, {medical_batch.max():.3f}]")

# Statistiques par image
print("\nAnalyse du lot:")
for i in range(medical_batch.shape[0]):
    img = medical_batch[i, 0]
    mean_intensity = img.mean()
    std_intensity = img.std()
    max_intensity = img.max()
    
    # Détection d'anomalie basée sur l'intensité maximale
    anomaly_detected = max_intensity > 0.8
    status = "ANOMALIE" if anomaly_detected else "NORMAL"
    
    print(f"Image {i+1}: Moyenne={mean_intensity:.3f}, Écart-type={std_intensity:.3f}, Max={max_intensity:.3f} [{status}]")

In [ ]:
# Visualisation du lot d'images médicales
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Lot d\'Images Médicales - Analyse Automatisée', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    if i < medical_batch.shape[0]:
        img = medical_batch[i, 0]
        im = ax.imshow(img, cmap='gray', vmin=0, vmax=1)
        
        # Classification automatique
        max_intensity = img.max().item()
        if max_intensity > 0.8:
            title_color = 'red'
            classification = 'ANOMALIE'
        else:
            title_color = 'green'
            classification = 'NORMAL'
        
        ax.set_title(f'Image {i+1}\n{classification}', color=title_color, fontweight='bold')
        ax.axis('off')
        
        # Ajouter une barre de couleur pour la première image
        if i == 0:
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

## 3. Transformations Médicales Standardisées

Les images médicales nécessitent des prétraitements spécifiques pour optimiser les performances des modèles d'IA.

In [ ]:
# Pipeline de transformations médicales
print("=== PIPELINE DE TRANSFORMATIONS MÉDICALES ===")

class MedicalImageProcessor:
    """
    Processeur d'images médicales avec transformations standardisées
    """
    
    def __init__(self, target_size=224, normalize_to_unit=True):
        self.target_size = target_size
        self.normalize_to_unit = normalize_to_unit
        
        # Transformations pour l'entraînement (avec augmentation)
        self.train_transforms = transforms.Compose([
            transforms.Resize((target_size, target_size)),
            transforms.RandomRotation(degrees=5),  # Rotation légère médicalement plausible
            transforms.RandomHorizontalFlip(p=0.1),  # Flip limité (asymétrie anatomique)
            transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Variation d'acquisition
            transforms.ToTensor(),
        ])
        
        # Transformations pour l'inférence (sans augmentation)
        self.inference_transforms = transforms.Compose([
            transforms.Resize((target_size, target_size)),
            transforms.ToTensor(),
        ])
    
    def normalize_medical_image(self, image_tensor):
        """
        Normalisation spécifique aux images médicales
        """
        if self.normalize_to_unit:
            # Normalisation min-max à [0,1]
            min_val = image_tensor.min()
            max_val = image_tensor.max()
            if max_val > min_val:
                image_tensor = (image_tensor - min_val) / (max_val - min_val)
        
        return image_tensor
    
    def process_batch(self, image_batch, mode='inference'):
        """
        Traite un lot d'images médicales
        """
        processed_images = []
        
        for i in range(image_batch.shape[0]):
            # Conversion en PIL Image pour les transformations
            img_array = image_batch[i, 0].numpy()
            img_pil = Image.fromarray((img_array * 255).astype(np.uint8), mode='L')
            
            # Application des transformations
            if mode == 'train':
                img_transformed = self.train_transforms(img_pil)
            else:
                img_transformed = self.inference_transforms(img_pil)
            
            # Normalisation médicale
            img_normalized = self.normalize_medical_image(img_transformed)
            processed_images.append(img_normalized)
        
        return torch.stack(processed_images)

# Test du processeur
processor = MedicalImageProcessor(target_size=224)

# Traitement pour l'inférence
processed_batch = processor.process_batch(medical_batch, mode='inference')
print(f"Lot original: {medical_batch.shape}")
print(f"Lot traité: {processed_batch.shape}")
print(f"Plage normalisée: [{processed_batch.min():.3f}, {processed_batch.max():.3f}]")

# Comparaison visuelle
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Comparaison: Avant et Après Traitement', fontsize=16)

for i in range(4):
    # Image originale
    axes[0, i].imshow(medical_batch[i, 0], cmap='gray')
    axes[0, i].set_title(f'Original {i+1}\n{medical_batch.shape[2]}x{medical_batch.shape[3]}')
    axes[0, i].axis('off')
    
    # Image traitée
    axes[1, i].imshow(processed_batch[i, 0], cmap='gray')
    axes[1, i].set_title(f'Traité {i+1}\n{processed_batch.shape[2]}x{processed_batch.shape[3]}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 4. Intégration HuggingFace pour Modèles Médicaux

HuggingFace propose une vaste collection de modèles pré-entraînés adaptés au domaine médical. Apprenons à les intégrer efficacement.

In [ ]:
# Chargement d'un modèle médical depuis HuggingFace
print("=== INTÉGRATION HUGGINGFACE POUR LE MÉDICAL ===")

# Exemple 1: Modèle de texte médical français
try:
    # Tokenizer et modèle pour le français médical
    model_name = "camembert-base"  # Base pour le français
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    print(f"Modèle chargé: {model_name}")
    print(f"Taille du vocabulaire: {len(tokenizer)}")
    
    # Test avec du texte médical français
    medical_texts = [
        "Patient présentant une dyspnée d'effort avec douleurs thoraciques.",
        "Radiographie pulmonaire montrant une opacité lobaire supérieure droite.",
        "Échographie abdominale révélant une hépatomégalie modérée.",
        "Électrocardiogramme normal, rythme sinusal régulier."
    ]
    
    print("\nTokenisation d'exemples médicaux:")
    for i, text in enumerate(medical_texts):
        # Tokenisation
        tokens = tokenizer.tokenize(text)
        input_ids = tokenizer.encode(text, return_tensors='pt')
        
        print(f"\nTexte {i+1}: {text[:50]}...")
        print(f"Nombre de tokens: {len(tokens)}")
        print(f"Premiers tokens: {tokens[:5]}")
        
        # Inférence rapide (embeddings)
        with torch.no_grad():
            outputs = model(input_ids)
            embeddings = outputs.last_hidden_state
            
        print(f"Dimensions embeddings: {embeddings.shape}")
        
        # Embedding moyen de la phrase (représentation vectorielle)
        sentence_embedding = embeddings.mean(dim=1).squeeze()
        print(f"Représentation vectorielle: {sentence_embedding.shape} (norme: {torch.norm(sentence_embedding):.3f})")

except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")
    print("Vérifiez votre connexion internet ou utilisez un modèle local.")

In [ ]:
# Exemple 2: Similarité sémantique entre termes médicaux
print("\n=== ANALYSE DE SIMILARITÉ MÉDICALE ===")

def compute_medical_similarity(texts, model, tokenizer):
    """
    Calcule la similarité cosinus entre textes médicaux
    """
    embeddings = []
    
    for text in texts:
        # Encodage et inférence
        input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=128)
        
        with torch.no_grad():
            outputs = model(input_ids)
            # Moyenne des embeddings comme représentation de la phrase
            embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
            embeddings.append(embedding)
    
    # Conversion en matrice
    embeddings = torch.stack(embeddings)
    
    # Calcul de similarité cosinus
    similarity_matrix = torch.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=2)
    
    return similarity_matrix

# Termes médicaux pour test de similarité
medical_terms = [
    "pneumonie",
    "infection pulmonaire",
    "cardiomégalie",
    "hypertrophie cardiaque",
    "fracture osseuse",
    "diabète sucré"
]

try:
    # Calcul des similarités
    similarity_matrix = compute_medical_similarity(medical_terms, model, tokenizer)
    
    # Visualisation
    plt.figure(figsize=(10, 8))
    sns.heatmap(similarity_matrix.numpy(), annot=True, cmap='YlOrRd', 
                xticklabels=medical_terms, yticklabels=medical_terms, fmt='.3f')
    plt.title('Matrice de Similarité Sémantique - Termes Médicaux')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Analyse des paires les plus similaires
    print("\nPaires de termes médicaux les plus similaires:")
    for i in range(len(medical_terms)):
        for j in range(i+1, len(medical_terms)):
            similarity = similarity_matrix[i, j].item()
            if similarity > 0.7:  # Seuil de similarité élevée
                print(f"'{medical_terms[i]}' ↔ '{medical_terms[j]}': {similarity:.3f}")

except Exception as e:
    print(f"Erreur lors du calcul de similarité: {e}")

## 5. Inférence avec Modèles Pré-entraînés

L'utilisation de modèles pré-entraînés est essentielle pour des applications médicales rapides et fiables.

In [ ]:
# Simuler l'utilisation d'un modèle de classification d'images médicales
print("=== INFÉRENCE AVEC MODÈLES PRÉ-ENTRAÎNÉS ===")

class MedicalImageClassifier:
    """
    Simulateur de modèle de classification d'images médicales
    En pratique, ceci serait un vrai modèle comme TorchXRayVision
    """
    
    def __init__(self, num_classes=5):
        self.num_classes = num_classes
        self.class_names = [
            "Normal",
            "Pneumonie",
            "Cardiomégalie", 
            "Œdème pulmonaire",
            "Pneumothorax"
        ]
        
        # Architecture simple pour démonstration
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
        
        self.classifier = nn.Linear(32, num_classes)
        
        # Initialisation simple
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialisation des poids du modèle"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
    
    def predict(self, images):
        """
        Prédiction sur un lot d'images
        """
        with torch.no_grad():
            features = self.features(images)
            logits = self.classifier(features)
            probabilities = torch.softmax(logits, dim=1)
            predictions = torch.argmax(probabilities, dim=1)
            
        return predictions, probabilities
    
    def get_prediction_summary(self, images):
        """
        Génère un résumé détaillé des prédictions
        """
        predictions, probabilities = self.predict(images)
        
        results = []
        for i in range(len(images)):
            pred_idx = predictions[i].item()
            pred_class = self.class_names[pred_idx]
            confidence = probabilities[i, pred_idx].item()
            
            # Distribution complète des probabilités
            prob_dist = {name: probabilities[i, j].item() 
                        for j, name in enumerate(self.class_names)}
            
            results.append({
                'image_id': i,
                'predicted_class': pred_class,
                'confidence': confidence,
                'probability_distribution': prob_dist
            })
        
        return results

# Instanciation et test du modèle
classifier = MedicalImageClassifier()
print(f"Modèle créé avec {sum(p.numel() for p in classifier.parameters())} paramètres")

# Prédiction sur notre lot d'images médicales
results = classifier.get_prediction_summary(processed_batch)

print("\nRésultats de classification:")
print("=" * 50)

for result in results:
    img_id = result['image_id']
    pred_class = result['predicted_class']
    confidence = result['confidence']
    
    print(f"\nImage {img_id + 1}:")
    print(f"  Diagnostic: {pred_class}")
    print(f"  Confiance: {confidence:.1%}")
    
    # Affichage des top 3 probabilités
    prob_dist = result['probability_distribution']
    sorted_probs = sorted(prob_dist.items(), key=lambda x: x[1], reverse=True)
    
    print(f"  Top 3 diagnostics:")
    for j, (class_name, prob) in enumerate(sorted_probs[:3]):
        print(f"    {j+1}. {class_name}: {prob:.1%}")

In [ ]:
# Visualisation des résultats de classification
fig, axes = plt.subplots(2, 4, figsize=(16, 10))
fig.suptitle('Classification Automatique d\'Images Médicales', fontsize=16, fontweight='bold')

for i, (result, ax) in enumerate(zip(results, axes.flat)):
    # Affichage de l'image
    ax.imshow(processed_batch[i, 0], cmap='gray')
    
    # Informations de classification
    pred_class = result['predicted_class']
    confidence = result['confidence']
    
    # Couleur selon la confiance
    if confidence > 0.7:
        title_color = 'green'
        conf_level = 'HAUTE'
    elif confidence > 0.5:
        title_color = 'orange'
        conf_level = 'MODÉRÉE'
    else:
        title_color = 'red'
        conf_level = 'FAIBLE'
    
    ax.set_title(f'Image {i+1}\n{pred_class}\nConfiance: {confidence:.1%} ({conf_level})', 
                color=title_color, fontweight='bold', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

# Statistiques globales de la classification
print("\n=== STATISTIQUES DE CLASSIFICATION ===")
predictions_count = {}
confidence_levels = {'HAUTE': 0, 'MODÉRÉE': 0, 'FAIBLE': 0}
total_confidence = 0

for result in results:
    pred_class = result['predicted_class']
    confidence = result['confidence']
    
    # Comptage des prédictions
    predictions_count[pred_class] = predictions_count.get(pred_class, 0) + 1
    
    # Niveaux de confiance
    if confidence > 0.7:
        confidence_levels['HAUTE'] += 1
    elif confidence > 0.5:
        confidence_levels['MODÉRÉE'] += 1
    else:
        confidence_levels['FAIBLE'] += 1
    
    total_confidence += confidence

print(f"Répartition des diagnostics:")
for diagnosis, count in predictions_count.items():
    percentage = (count / len(results)) * 100
    print(f"  {diagnosis}: {count} cas ({percentage:.1f}%)")

print(f"\nNiveaux de confiance:")
for level, count in confidence_levels.items():
    percentage = (count / len(results)) * 100
    print(f"  {level}: {count} cas ({percentage:.1f}%)")

print(f"\nConfiance moyenne: {total_confidence / len(results):.1%}")

## 6. Optimisation et Déploiement

Pour une utilisation clinique efficace, l'optimisation des modèles PyTorch est cruciale.

In [ ]:
# Techniques d'optimisation pour modèles médicaux
print("=== OPTIMISATION POUR DÉPLOIEMENT CLINIQUE ===")

import time

class OptimizedMedicalClassifier:
    """
    Version optimisée du classificateur médical pour déploiement
    """
    
    def __init__(self, original_model):
        self.original_model = original_model
        self.optimized_model = None
        self.class_names = original_model.class_names
    
    def optimize_for_inference(self):
        """
        Optimise le modèle pour l'inférence rapide
        """
        # Mode évaluation
        self.original_model.eval()
        
        # Optimisation avec TorchScript
        example_input = torch.randn(1, 1, 224, 224)
        
        try:
            # Création du modèle TorchScript
            self.optimized_model = torch.jit.trace(self.original_model, example_input)
            self.optimized_model.eval()
            print("Optimisation TorchScript: SUCCÈS")
        except Exception as e:
            print(f"Optimisation TorchScript: ÉCHEC ({e})")
            self.optimized_model = self.original_model
        
        return self.optimized_model
    
    def benchmark_performance(self, test_batch, num_runs=100):
        """
        Compare les performances avant et après optimisation
        """
        print(f"\nBenchmark de performance ({num_runs} exécutions):")
        
        # Test du modèle original
        start_time = time.time()
        for _ in range(num_runs):
            with torch.no_grad():
                _ = self.original_model.predict(test_batch[:1])  # 1 image à la fois
        original_time = time.time() - start_time
        
        # Test du modèle optimisé
        if self.optimized_model:
            start_time = time.time()
            for _ in range(num_runs):
                with torch.no_grad():
                    # Pour TorchScript, appel direct
                    features = self.optimized_model.features(test_batch[:1])
                    _ = self.optimized_model.classifier(features)
            optimized_time = time.time() - start_time
            
            speedup = original_time / optimized_time
            
            print(f"  Modèle original: {original_time:.3f}s ({original_time/num_runs*1000:.1f}ms par image)")
            print(f"  Modèle optimisé: {optimized_time:.3f}s ({optimized_time/num_runs*1000:.1f}ms par image)")
            print(f"  Accélération: {speedup:.2f}x")
        else:
            print(f"  Modèle original: {original_time:.3f}s ({original_time/num_runs*1000:.1f}ms par image)")
            print("  Pas d'optimisation disponible")
    
    def save_for_production(self, save_path):
        """
        Sauvegarde le modèle optimisé pour production
        """
        if self.optimized_model:
            torch.jit.save(self.optimized_model, save_path + "_optimized.pt")
            print(f"Modèle optimisé sauvegardé: {save_path}_optimized.pt")
        
        # Sauvegarde des métadonnées
        metadata = {
            'class_names': self.class_names,
            'input_size': [224, 224],
            'num_classes': len(self.class_names),
            'optimization': 'TorchScript' if self.optimized_model else 'None'
        }
        
        torch.save(metadata, save_path + "_metadata.pt")
        print(f"Métadonnées sauvegardées: {save_path}_metadata.pt")

# Test d'optimisation
optimizer = OptimizedMedicalClassifier(classifier)
optimized_model = optimizer.optimize_for_inference()

# Benchmark de performance
optimizer.benchmark_performance(processed_batch, num_runs=50)

# Sauvegarde pour production
optimizer.save_for_production("./medical_classifier")

In [ ]:
# Analyse de l'utilisation mémoire
print("\n=== ANALYSE DE L'UTILISATION MÉMOIRE ===")

def analyze_memory_usage(model, input_batch):
    """
    Analyse l'utilisation mémoire du modèle
    """
    # Taille du modèle
    model_size = sum(p.numel() * p.element_size() for p in model.parameters())
    print(f"Taille du modèle: {model_size / 1024**2:.1f} MB")
    
    # Taille des paramètres
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Nombre de paramètres: {num_params:,}")
    
    # Mémoire d'entrée
    input_memory = input_batch.element_size() * input_batch.numel()
    print(f"Mémoire d'entrée (batch de {input_batch.shape[0]}): {input_memory / 1024**2:.1f} MB")
    
    # Estimation mémoire totale
    total_memory = model_size + input_memory
    print(f"Estimation mémoire totale: {total_memory / 1024**2:.1f} MB")
    
    return {
        'model_size_mb': model_size / 1024**2,
        'num_params': num_params,
        'input_memory_mb': input_memory / 1024**2,
        'total_memory_mb': total_memory / 1024**2
    }

memory_stats = analyze_memory_usage(classifier, processed_batch)

# Recommandations de déploiement
print("\n=== RECOMMANDATIONS DE DÉPLOIEMENT ===")

total_memory = memory_stats['total_memory_mb']

if total_memory < 100:
    deployment_tier = "MOBILE/EDGE"
    recommendations = [
        "Déploiement possible sur appareils mobiles",
        "Utilisation en consultation sans connexion",
        "Intégration dans équipements médicaux portables"
    ]
elif total_memory < 500:
    deployment_tier = "SERVEUR LOCAL"
    recommendations = [
        "Déploiement sur serveur hospitalier",
        "Intégration PACS locale",
        "Traitement en temps réel des examens"
    ]
else:
    deployment_tier = "CLOUD/GPU"
    recommendations = [
        "Déploiement cloud nécessaire",
        "Utilisation GPU recommandée",
        "Traitement par lots optimisé"
    ]

print(f"Niveau de déploiement recommandé: {deployment_tier}")
print("Recommandations:")
for rec in recommendations:
    print(f"  • {rec}")

# Estimation du débit
images_per_second = 1000 / (memory_stats['total_memory_mb'] / 10)  # Estimation simplifiée
print(f"\nDébit estimé: {images_per_second:.0f} images/seconde")
print(f"Capacité journalière estimée: {images_per_second * 3600 * 8:.0f} images/jour (8h)")

## Résumé et Applications Pratiques

### Compétences Acquises

Dans ce notebook, vous avez appris à:

1. **Manipuler des données médicales** avec les tenseurs PyTorch
   - Calculs vectorisés sur cohortes de patients
   - Analyses statistiques et corrélations cliniques
   - Traitement par lots d'images médicales

2. **Intégrer des modèles HuggingFace** pour le médical
   - Chargement de modèles français pré-entraînés
   - Tokenisation et embeddings de textes médicaux
   - Analyse de similarité sémantique

3. **Effectuer l'inférence** avec modèles pré-entraînés
   - Classification automatique d'images médicales
   - Interprétation des probabilités et confiance
   - Génération de rapports automatisés

4. **Optimiser pour le déploiement** clinique
   - TorchScript pour accélération
   - Analyse d'utilisation mémoire
   - Recommandations de déploiement

### Applications Cliniques Directes

Ces compétences PyTorch vous permettront de:
- **Intégrer des modèles IA** dans vos workflows médicaux
- **Analyser des cohortes** de patients efficacement
- **Traiter des images médicales** par lots
- **Déployer des solutions** dans votre environnement clinique

### Prochaine Étape

Le prochain notebook vous enseignera la **classification de textes médicaux en français** avec des modèles spécialisés, en utilisant les bases PyTorch que vous venez d'acquérir.